In [1]:
import os
import glob
import cuvis
import cuvis_ai
import cuvis_ai.data
from PIL import Image as Img
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
# Download a publicly available dataset
base_path = "./cuvis_ai_video"
os.mkdir(base_path)
data_down = cuvis_ai.data.PublicDataSets()
data_down.download_dataset("Aquarium", download_path=base_path)

In [3]:
# Let's look at a single example
# BASEPATH should be the location of the BananaNet training data
cubes = glob.glob(f'{base_path}/*.cu3s')
data = cuvis.SessionFile(cubes[0]).get_measurement(0)
sample_cube = data.data.get('cube').array
waves = data.data.get('cube').wavelength

In [ ]:
# Show RGB image
plt.subplot(1, 2, 1)
plt.imshow(sample_cube[:,:,10])
plt.scatter([50],[50],[15],['red'])
plt.scatter([50],[30],[15],['orange'])
plt.scatter([50],[150],[15],['green'])
plt.scatter([150],[150],[15],['magenta'])
plt.title('Single Channel Representation')
plt.subplot(1, 2, 2)
plt.plot(waves, sample_cube[50,50,:], 'red', label='Water')
plt.plot(waves, sample_cube[30,50,:], 'orange', label='Fish')
plt.plot(waves, sample_cube[150,50,:], 'green', label='Plant')
plt.plot(waves, sample_cube[150,150,:], 'magenta', label='Plant 2')
plt.title('Spectra Representation')
plt.legend()
plt.tight_layout()
plt.show()

In [5]:
# Define number of parameters useful for understanding our classes
number_of_components = 6
number_of_classes = 4

In [6]:
# Define SAM node
sam = cuvis_ai.distance.SpectralAngle(ref_spectra=[
    sample_cube[50,50,:],
    sample_cube[30,50,:],
    sample_cube[150,50,:],
    sample_cube[150,150,:]
])

In [9]:
# Distance decider node
decider = cuvis_ai.deciders.MultiClassDecider(number_of_classes, use_min=True)

In [ ]:
# Define and construct graph
# This will throw an initialization warning "Unsatisfied dimensionality constraint"
# This is expected behavior
graph = cuvis_ai.pipeline.Graph("DemoGraph")
graph.add_base_node(sam)
graph.add_edge(sam, decider)

In [ ]:
# Define unlabeled dataset
data = cuvis_ai.data.CuvisDataSet(base_path)

In [ ]:
# Fit the graph using the first image
graph.fit(*data[0:1])

In [ ]:
# Generate gifs of session file classification
for i in tqdm.tqdm(range(len(data))):
    res_show, _, _ = graph.forward(*data[i:i+1])
    plt.figure()
    plt.imshow(res_show[0,:,:,:])  # Example plot
    plt.title('CUVIS AI Classification')
    filename = os.path.join(base_path
    , f"frame_{i}.png")
    plt.axis('off')
    plt.savefig(filename)
    plt.close()

In [14]:
# Save as an animated GIF

# Collect all the frames
frames = []
for i in range(len(data)):
    filename = os.path.join(base_path, f"frame_{i}.png")
    frames.append(Img.open(filename))
    
# Define save path
gif_path = os.path.join(base_path, "cuvis_fishtank_example_sam.gif")
frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=50,  # Duration between frames in milliseconds
    loop=0  # Loop forever
)


In [ ]:
display(Image(filename=gif_path))